# Multi-Layer Perceptrons / Dense Layers

In [ ]:
# Setup
import tensorflow as tf
import keras

import numpy as np
import datetime

import os
import sys
import imp

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
import models as M
import utils as U

imp.reload(M)
imp.reload(U)

### Constants

In [ ]:
EPOCH = 30
BATCH = 128

DATADIR = "/Users/mghifary/Work/Code/AI/data"
MODELDIR = "/Users/mghifary/Work/Code/AI/models"

### Load and preprocess dataset

In [ ]:
# Load dataset
data_path = os.path.join(DATADIR, "mnist.npz")
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data(data_path)
x_train = x_train.astype("float32") / 255.
x_test = x_test.astype("float32") / 255.

# Reshape
x_train = np.expand_dims(x_train, axis=-1) 
x_test = np.expand_dims(x_test, axis=-1)

(n_train, dx1, dx2, c) = x_train.shape
n_test = x_test.shape[0]

ds_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
ds_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))

# Shuffle and batch
ds_train = ds_train.shuffle(buffer_size=1024).batch(BATCH)
ds_test = ds_test.batch(BATCH)

### Visualize data

In [ ]:
# Visualize training samples
U.visualize_dataset(ds_train, title="MNIST Training", n_samples=25)

In [ ]:
# Visualize test samples
U.visualize_dataset(ds_test, title="MNIST Testing", n_samples=25)

### Build NN model

In [ ]:
# Sequential model
model = M.MLPSeq(num_classes=10)
model.build(input_shape=(None, dx1, dx2, c))
model.summary()

In [ ]:
# # Functional API
# model = M.make_mlp(input_shape=(dx1, dx2, c), num_classes=10)
# model.summary()

In [ ]:
# # Subclassing API
# model = M.MLP(num_classes=10)
# model.build(input_shape=(None, dx1, dx2, c))
# model.summary()

#### Train model

In [ ]:
# Loss and optimizer
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
accuracy = keras.metrics.SparseCategoricalAccuracy()
optimizer = keras.optimizers.legacy.Adam(learning_rate=3e-4)

In [ ]:
# Define callback for Tensorboard and Model Checkpoint
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback = keras.callbacks.TensorBoard(
    log_dir="./logs/mlp-mnist/" + current_time,
    update_freq="epoch"
)

checkpoint_dir = os.path.join(MODELDIR, f"mlp-mnist-{current_time}")

if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

checkpoint_filepath = os.path.join(checkpoint_dir, "weights-{epoch:02d}-{val_loss:.2f}.hdf5")
modelcp_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True
)

In [ ]:
# Compile and run training
model.compile(optimizer=optimizer, loss=loss_fn, metrics=[accuracy])
model.fit(ds_train, epochs=EPOCH, validation_data=ds_test, callbacks=[tensorboard_callback, modelcp_callback])